# Interactive Audit Testing

This notebook allows for real-time testing of the IFRS9 Automated Auditor without using web ports.

In [ ]:
# Cell 1: Load environment and initialize RcmAuditor
import os
from dotenv import load_dotenv
from rcm_engine import RcmAuditor
from config import CONFIG

load_dotenv()

# Initialize Auditor
auditor = RcmAuditor()
print("RcmAuditor initialized.")

# Ensure documents are indexed
auditor.initialize_rag()
print("RAG Index ready.")

In [ ]:
# Cell 2: Define a specific test question manually
test_question = "How is the PD 12-month calculated?"
print(f"Test Question: {test_question}")

In [ ]:
# Cell 3: Run the retrieval step only and print retrieved PDF text chunks
print("Retrieving top 5 chunks...")
retrieved_docs = auditor.rag_engine.retrieve(test_question, k=5)

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Chunk {i+1} (Page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:500] + "...") # Print first 500 chars

In [ ]:
# Cell 4: Run the full process_row logic and print AI Answer AND Critique Score

# Mock a row data structure
mock_row = {
    'Control Reference': 'Test-Ref-001',
    'Test Procedure': test_question
}

print("Running full process_row...")
result = auditor.process_row(mock_row)

print("\n=== AI Answer ===")
print(result['AI_Answer'])

print("\n=== Validation ===")
print(f"Score: {result['Validation_Score']}")
print(f"Hallucination: {result['Hallucination_Flag']}")
print(f"Reasoning: {result['Validation_Reasoning']}")